<a href="https://colab.research.google.com/github/SandyCOG/Aviation-Weather-Forecasting-Using-METAR-Data/blob/main/Aviation_Weather_Forecasting_Using_METAR_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install metar


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from metar import Metar

In [8]:
df = pd.read_csv("/content/Aviation Weather Forecasting Using METAR Data Set.txt", sep=",")
df.head()

,airport_id,date,metar
0,KMIA,2014-01-01,KMIA 010053Z 04009G15KT 10SM OVC025 23/18 A302...
1,KMIA,2014-01-01,KMIA 010153Z 04009G15KT 10SM OVC023 23/18 A302...
2,KMIA,2014-01-01,KMIA 010253Z 05009G19KT 10SM -RA OVC025 23/18 ...
3,KMIA,2014-01-01,KMIA 010353Z 07010KT 10SM OVC028 23/17 A3021 R...
4,KMIA,2014-01-01,KMIA 010453Z 07010KT 10SM OVC028 24/18 A3019 R...


In [4]:

obs = Metar.Metar('METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD AND NW -N-E MOV NE P0013 T02270215')
obs.string()

'station: KEWR\ntype: routine report, cycle 19 (automatic report)\ntime: Sun Feb 11 18:51:00 2024\ntemperature: 22.7 C\ndew point: 21.5 C\nwind: variable at 3 knots, gusting to 19 knots\npeak wind: WNW at 28 knots at 18:17\nwind shift: 18:12\nvisibility: 2 miles\nvisual range: on runway 04R, from 3000 to greater than 6000 feet\npressure: 1011.5 mb\nweather: thunderstorm with rain; mist\nsky: a few clouds at 1500 feet\n     broken cumulonimbus at 4000 feet\n     broken clouds at 6500 feet\n     overcast at 20000 feet\nsea-level pressure: 1011.4 mb\n1-hour precipitation: 0.13in\nremarks:\n- Automated station (type 2)\n- peak wind 28kt from 290 degrees at 18:17\n- wind shift at 18:12\n- frequent lightning (intracloud,cloud-to-cloud,cloud-to-ground)\n- thunderstorm overhead and NW\n- TSB05RAB22 -N-E MOV NE\nMETAR: METAR KEWR 111851Z VRB03G19KT 2SM R04R/3000VP6000FT TSRA BR FEW015 BKN040CB BKN065 OVC200 22/22 A2987 RMK AO2 PK WND 29028/1817 WSHFT 1812 TSB05RAB22 SLP114 FRQ LTGICCCCG TS OHD 

In [37]:
# Function to parse METAR string and return a dictionary
def parse_metar(metar_string):
    try:
        obs = Metar.Metar(metar_string)
    except Metar.ParserError as e:
        # Handle parsing errors, you can print the error message for debugging
        print(f"Error parsing METAR: {e}")
        return None

    def get_attribute_value(attribute):
        if hasattr(obs, attribute) and getattr(obs, attribute) is not None:
            return getattr(obs, attribute).string()
        return None

    return {
        'station': obs.station_id,
        'type': obs.report_type(),
        'time': obs.time,
        'temperature': get_attribute_value('temp'),
        'dew_point': get_attribute_value('dewpt'),
        'wind': obs.wind(),
        'peak_wind': obs.peak_wind(),
        'wind_shift': obs.wind_shift(),
        'visibility': get_attribute_value('vis'),
        'pressure': get_attribute_value('press'),
        'weather': obs.present_weather(),
        'sky': obs.sky_conditions(),
        'sea_level_pressure': get_attribute_value('sea_level_pressure'),
        'remarks': obs.remarks(),
        'metar': metar_string
    }



# Apply the parse_metar function to the "metar" column
df_parsed = df['metar'].apply(parse_metar).apply(pd.Series)

# Concatenate the parsed information with the original DataFrame
df = pd.concat([df, df_parsed], axis=1)

# Display the resulting DataFrame
df

Error parsing METAR: Unparsed groups in body '1822LVA1356 FF KZCRZQZX KZMAZQZX MKJKZPZX MKJKZQZX MPTOZQZX MPZLZQZX MUFHZQZX MUHAZPZX SPIMZQZA (ARR-ABX2260-KMIA-SPJC2200)' while processing 'SCGE 1822LVA1356 182200 FF KZCRZQZX KZMAZQZX MKJKZPZX MKJKZQZX MPTOZQZX MPZLZQZX MUFHZQZX MUHAZPZX 182200 SPIMZQZA (ARR-ABX2260-KMIA-SPJC2200)'
Error parsing METAR: Unparsed groups in body 'LVA0485 FF KFLLYFYX KMIAYFYX KZCRZQZX KZMAZQZX MHCCZQZX MKJKZQZX MMFRZQZX MPZLZQZX MUFHZQZX 251101 SEQMZPZX (FPL-AAL932-IS -B38M/M-SDE3GHIJ4RWXZ/B1L -SEQM1138 -N0462F340 PALAD1A PALAD UL318 VAMOS UL318 ESV DCT UGUPI UL7' while processing 'SPJR 251100Z VRB02KT CAVOK LVA0485 251101 FF KFLLYFYX KMIAYFYX KZCRZQZX KZMAZQZX MHCCZQZX MKJKZQZX MMFRZQZX MPZLZQZX MUFHZQZX 251101 SEQMZPZX (FPL-AAL932-IS -B38M/M-SDE3GHIJ4RWXZ/B1L -SEQM1138 -N0462F340 PALAD1A PALAD UL318 VAMOS UL318 ESV DCT UGUPI UL7'


<ipython-input-37-b9c7eabfcf45>:36: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_parsed = df['metar'].apply(parse_metar).apply(pd.Series)
<ipython-input-37-b9c7eabfcf45>:36: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_parsed = df['metar'].apply(parse_metar).apply(pd.Series)


,airport_id,date,metar,station,type,time,temperature,dew_point,wind,peak_wind,wind_shift,visibility,pressure,weather,sky,sea_level_pressure,remarks,metar
0,KMIA,2014-01-01,KMIA 010053Z 04009G15KT 10SM OVC025 23/18 A302...,KMIA,"routine report, cycle 1 (automatic report)",2024-02-01 00:53:00,23.3 C,18.3 C,"NE at 9 knots, gusting to 15 knots",missing,missing,10 miles,30.21 inches,,overcast at 2500 feet,NaN,Automated station (type 2),KMIA 010053Z 04009G15KT 10SM OVC025 23/18 A302...
1,KMIA,2014-01-01,KMIA 010153Z 04009G15KT 10SM OVC023 23/18 A302...,KMIA,"routine report, cycle 2 (automatic report)",2024-02-01 01:53:00,23.3 C,18.3 C,"NE at 9 knots, gusting to 15 knots",missing,missing,10 miles,30.21 inches,,overcast at 2300 feet,NaN,Automated station (type 2),KMIA 010153Z 04009G15KT 10SM OVC023 23/18 A302...
2,KMIA,2014-01-01,KMIA 010253Z 05009G19KT 10SM -RA OVC025 23/18 ...,KMIA,"routine report, cycle 3 (automatic report)",2024-02-01 02:53:00,22.8 C,17.8 C,"NE at 9 knots, gusting to 19 knots",missing,missing,10 miles,30.23 inches,light rain,overcast at 2500 feet,NaN,Automated station (type 2); 3-hr pressure chan...,KMIA 010253Z 05009G19KT 10SM -RA OVC025 23/18 ...
3,KMIA,2014-01-01,KMIA 010353Z 07010KT 10SM OVC028 23/17 A3021 R...,KMIA,"routine report, cycle 4 (automatic report)",2024-02-01 03:53:00,23.3 C,17.2 C,ENE at 10 knots,missing,missing,10 miles,30.21 inches,,overcast at 2800 feet,NaN,Automated station (type 2),KMIA 010353Z 07010KT 10SM OVC028 23/17 A3021 R...
4,KMIA,2014-01-01,KMIA 010453Z 07010KT 10SM OVC028 24/18 A3019 R...,KMIA,"routine report, cycle 5 (automatic report)",2024-02-01 04:53:00,23.9 C,17.8 C,ENE at 10 knots,missing,missing,10 miles,30.19 inches,,overcast at 2800 feet,NaN,Automated station (type 2),KMIA 010453Z 07010KT 10SM OVC028 24/18 A3019 R...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48177,KMIA,2019-07-06,KMIA 060853Z 11006KT 10SM FEW020 SCT220 28/23 ...,KMIA,"routine report, cycle 9 (automatic report)",2024-02-06 08:53:00,28.3 C,22.8 C,ESE at 6 knots,missing,missing,10 miles,30.06 inches,,a few clouds at 2000 feet; scattered clouds at...,NaN,Automated station (type 2); 3-hr pressure chan...,KMIA 060853Z 11006KT 10SM FEW020 SCT220 28/23 ...
48178,KMIA,2019-07-06,KMIA 060953Z 10006KT 10SM FEW025 FEW040 SCT250...,KMIA,"routine report, cycle 10 (automatic report)",2024-02-06 09:53:00,28.9 C,22.8 C,E at 6 knots,missing,missing,10 miles,30.07 inches,,a few clouds at 2500 feet; a few clouds at 400...,NaN,Automated station (type 2); occasional lightni...,KMIA 060953Z 10006KT 10SM FEW025 FEW040 SCT250...
48179,KMIA,2019-07-06,KMIA 061053Z 12006KT 10SM FEW025 FEW040 BKN250...,KMIA,"routine report, cycle 11 (automatic report)",2024-02-06 10:53:00,28.3 C,23.3 C,ESE at 6 knots,missing,missing,10 miles,30.08 inches,,a few clouds at 2500 feet; a few clouds at 400...,NaN,Automated station (type 2),KMIA 061053Z 12006KT 10SM FEW025 FEW040 BKN250...
48180,KMIA,2019-07-06,KMIA 061153Z 12005KT 8SM SCT027CB SCT060 BKN15...,KMIA,"routine report, cycle 12 (automatic report)",2024-02-06 11:53:00,28.9 C,23.9 C,ESE at 5 knots,missing,missing,8 miles,30.09 inches,,scattered cumulonimbus at 2700 feet; scattered...,NaN,Automated station (type 2); 3-hr pressure chan...,KMIA 061153Z 12005KT 8SM SCT027CB SCT060 BKN15...
